In [13]:
#Imports
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

load_dotenv()
%matplotlib inline

In [2]:
api = os.getenv("LUNARCRUSHKEY")

In [26]:
BTCNews= "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,dot,xmr,xlm&sources=news,urls&limit=50000&type=chronological"

BTCNewsRequest = requests.get(BTCNews)
BTCNewsData = BTCNewsRequest.json()


In [27]:
#bitcoin (btc) | litecoin (ltc) | polkadot (dot) | monero (xmr) | stellar lumens (xlm)
#BTCNewsData["data"][1200]

In [31]:
sentiments = []

for article in BTCNewsData["data"]:
    try:
        text = article["description"]
        date = article["time"]
        symbol = article["symbol"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        sentiments.append({
            "Date": date,
            "Ticker": symbol,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            #"Text": text 
        })
        
    except AttributeError:
        pass 

# Create DataFrame
BTCNewsDF = pd.DataFrame(sentiments)
BTCNewsDF.tail(80)

,Date,Ticker,Compound,Positive,Negative,Neutral
2898,1607459700,XMR,-0.4404,0.000,0.094,0.906
2899,1607428800,DOT,0.0000,0.000,0.000,1.000
2900,1607391705,LTC,0.6249,0.188,0.000,0.812
2901,1607121776,BTC,0.2500,0.150,0.000,0.850
2902,1607110470,BTC,-0.4019,0.000,0.231,0.769
...,...,...,...,...,...,...
2973,1491307200,XMR,-0.8768,0.034,0.294,0.672
2974,1429717212,XLM,0.2263,0.079,0.000,0.921
2975,1403908140,BTC,0.2023,0.113,0.077,0.811
2976,1387211177,BTC,0.4019,0.137,0.000,0.863


In [6]:
BTCSocials= "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=BTC&limit=5000000&sources=twitter,reddit&type=chronological"

BTCSMRequest = requests.get(BTCSocials)
BTCSMData = BTCSMRequest.json()

In [7]:
#BTCSMData["data"][200]

In [8]:
sentiments = []

for article in BTCSMData["data"]:
    try:
        text = article["body"]
        date = article["time"]
        symbol = article["symbol"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        sentiments.append({
            "Date": date,
            "Ticker": symbol,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            #"Text": text 
        })
        
    except AttributeError:
        pass 

# Create DataFrame
BTCSMDF = pd.DataFrame(sentiments)
BTCSMDF.tail()

,Date,Ticker,Compound,Positive,Negative,Neutral
1207,1607827028,BTC,0.9859,0.192,0.068,0.740
1208,1607824282,BTC,0.4404,0.223,0.157,0.620
1209,1607824201,BTC,0.9880,0.125,0.025,0.850
1210,1607821399,BTC,-0.3612,0.099,0.143,0.758
1211,1607818095,BTC,0.0000,0.000,0.000,1.000


In [19]:
BTC_Sentiment = pd.concat([BTCNewsDF,BTCSMDF], axis="rows", ignore_index=True)
BTC_Sentiment['Date'] = pd.to_datetime(BTC_Sentiment['Date'],unit='s')
#BTC_Sentiment['Date'] = pd.to_datetime(BTC_Sentiment['Date']).dt.date
BTC_Sentiment = BTC_Sentiment.groupby(['Date','Ticker'],as_index=False).mean()
BTC_Sentiment = BTC_Sentiment.sort_values(by=['Date'], ascending=False,ignore_index=True)
BTC_Sentiment.set_index('Date', inplace=True, drop=True)
#BTC_Sentiment.reset_index(drop=True, inplace=True)
#BTC_Sentiment.reset_index(inplace=True)
#BTC_Sentiment.columns = ['Date', 'Ticker', 'Compound', 'Positive', 'Negative', 'Neutral']

BTC_Sentiment.tail()

,Ticker,Compound,Positive,Negative,Neutral
Date,,,,,
2017-12-29 01:14:33,BTC,0.7501,0.242,0.000,0.758
2017-12-28 09:02:21,BTC,0.2732,0.123,0.000,0.877
2014-06-27 22:29:00,BTC,0.2023,0.113,0.077,0.811
2013-12-16 16:26:17,BTC,0.4019,0.137,0.000,0.863
2013-03-07 12:00:00,BTC,-0.5994,0.000,0.157,0.843


In [17]:
# create a list of our conditions
conditions = [
    (BTC_Sentiment['Compound'] < 0),
    (BTC_Sentiment['Compound'] == 0),
    (BTC_Sentiment['Compound'] > 0)
    ]

# create a list of the values we want to assign for each condition
values = ['-1', '0', '1']

# create a new column and use np.select to assign values to it using our lists as arguments
BTC_Sentiment['Sentiment Signal'] = np.select(conditions, values)

# display updated DataFrame
BTC_Sentiment.head()

,Ticker,Compound,Positive,Negative,Neutral,Sentiment Signal
Date,,,,,,
2021-01-12 22:19:46,BTC,0.0000,0.000,0.0,1.000,0
2021-01-12 22:16:02,BTC,0.7269,0.244,0.0,0.756,1
2021-01-12 22:08:02,BTC,0.0000,0.000,0.0,1.000,0
2021-01-12 22:04:27,BTC,0.0000,0.000,0.0,1.000,0
2021-01-12 21:35:09,BTC,0.0000,0.000,0.0,1.000,0


In [18]:
btc_filldates = BTC_Sentiment.copy()
btc_filldates = btc_filldates.resample('D').bfill()

btc_filldates = btc_filldates.sort_values(by=['Date'], ascending=False)

btc_filldates.head()

,Ticker,Compound,Positive,Negative,Neutral,Sentiment Signal
Date,,,,,,
2021-01-12,BTC,0.136600,0.037500,0.000,0.962500,1
2021-01-11,BTC,0.440400,0.108000,0.000,0.892000,1
2021-01-10,BTC,-0.102700,0.000000,0.055,0.945000,-1
2021-01-09,BTC,0.190967,0.052667,0.000,0.947333,1
2021-01-08,BTC,0.045367,0.110667,0.125,0.764333,1
